# 🍊🗿제주 특산물 가격 예측

In [350]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [351]:
import pandas as pd
import numpy as np
import pyproj
import matplotlib.pyplot as plt
import re

In [352]:
path1 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/train.csv'
path2 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/international_trade.csv'
path3 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/sample_submission.csv'
path4 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/test.csv'
path5 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/df_holiday.csv'
path6 = '/content/drive/MyDrive/셩 코딩 연습/data/jeju/df_holiday2.csv'

In [387]:
#df = pd.read_csv(path1)
df_i = pd.read_csv(path2)  #수출입 정보
df_s = pd.read_csv(path3)
df_t = pd.read_csv(path4)

--------

## 🍏 LSTM

### 1. Initialization

In [354]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [355]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [388]:
'''
가중치를 임의의 값으로 초기화 하여
학습을 수행하는데, 실험을 동일하게 진행하기 위해서는
난수를 동일하게 생성해야한다.
Pytorch 에서 랜덤 시드를 고정하기 위해 -> manual_seed() 사용
'''
torch.manual_seed(0)

#gpu 사용 가능 환경인지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [389]:
#파라미터 초기화
seq_length = 7 #시퀸스 데이터 길이
data_dim = 6  #Feature 개수
hidden_dim = 10 #은닉층 차원
output_dim = 1  #출력 차원
learning_rate = 0.01  #학습률
epochs = 500  #학습 횟수
batch_size = 100  #배치 크기

### 2. Data Preprocessing

In [390]:
df = pd.read_csv(path6).drop(columns='ID')
df

,timestamp,item,corporation,location,supply,price,year,month,day,holiday
0,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1
1,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,0
2,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,0
3,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,0
4,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,1
...,...,...,...,...,...,...,...,...,...,...
59392,2023-03-02,BC,E,J,2984.0,3144.0,2023,3,2,0
59393,2023-03-03,BC,E,J,4920.0,3045.0,2023,3,3,0
59394,2023-03-01,BC,E,S,2160.0,3816.0,2023,3,1,0
59395,2023-03-02,BC,E,S,1152.0,3321.0,2023,3,2,0


In [391]:
#인코딩
category_columns = ['item','corporation','location']
df[category_columns] = df[category_columns].astype('category')
df_t[category_columns] = df_t[category_columns].astype('category')

df[category_columns] = df[category_columns].apply(lambda x : x.cat.codes)
df_t[category_columns] = df_t[category_columns].apply(lambda x : x.cat.codes)

In [392]:
df_t['year'] = df_t['timestamp'].apply(lambda x : int(x[0:4]))
df_t['month'] = df_t['timestamp'].apply(lambda x : int(x[5:7]))
df_t['day'] = df_t['timestamp'].apply(lambda x : int(x[8:10]))

In [393]:
df_t

,ID,timestamp,item,corporation,location,year,month,day
0,TG_A_J_20230304,2023-03-04,4,0,0,2023,3,4
1,TG_A_J_20230305,2023-03-05,4,0,0,2023,3,5
2,TG_A_J_20230306,2023-03-06,4,0,0,2023,3,6
3,TG_A_J_20230307,2023-03-07,4,0,0,2023,3,7
4,TG_A_J_20230308,2023-03-08,4,0,0,2023,3,8
...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,3,5,0,2023,3,27
1088,RD_F_J_20230328,2023-03-28,3,5,0,2023,3,28
1089,RD_F_J_20230329,2023-03-29,3,5,0,2023,3,29
1090,RD_F_J_20230330,2023-03-30,3,5,0,2023,3,30


In [395]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    59397 non-null  object 
 1   item         59397 non-null  int8   
 2   corporation  59397 non-null  int8   
 3   location     59397 non-null  int8   
 4   supply       59397 non-null  float64
 5   price        59397 non-null  float64
 6   year         59397 non-null  int64  
 7   month        59397 non-null  int64  
 8   day          59397 non-null  int64  
 9   holiday      59397 non-null  int64  
dtypes: float64(2), int64(4), int8(3), object(1)
memory usage: 3.3+ MB


In [396]:
trainX = df.drop(columns=['timestamp', 'supply', 'price'])
trainY = df['price']

testX = df_t.drop(columns=['ID', 'timestamp'])

In [399]:
X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size=0.2)

In [400]:
#학습용 데이터 셋 생성
#Sequence data(순서가 있는 데이터 예를들어 시계열, 텍스트 데이터) 만들어서 numpy array로 반환
def build_dataset(data, seq_len):
  dataX = []  #리스트 생성
  dataY = []
  for i in range(len(data)-seq_len):
    x = data[i:i+seq_len, :]
    y = data[i+seq_len,[-1]]
    dataX.append(x) #리스트 끝에 새로운 요소 추가
    dataY.append(y)
  return np.array(dataX), np.array(dataY)

In [406]:
#TensorDataset : 학습 데이터 X 와 레이블 Y 를 묶어 놓는 컨테이너
dataset = (trainX, trainY)

#DataLoader 를 통해 배치 크기, 데이터를 섞을지 여부들을 결정한당
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

### 3. LSTM 설계

In [407]:
#lstm class

class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, seq_len, output_dim, Layers):
  #모델에 필요한 값 초기화
    super(LSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.seq_len = seq_len
    self.output_dim = output_dim
    self.Layers = Layers
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers = Layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim, bias=True)

  #은닉층 초기화하는 함수
  #매번 새로운 학습을 하여 최적의 값을 찾기 위한 방안
  def reset_hidden_state(self): #hidden state 초기화
    self.hidden =(
        torch.zeros(self.Layers, self.seq_len, self.hidden_dim),
        torch.zeros(self.Layers, self.seq_len, self.hidden_dim)
    )
  #순전파 함수
  def forward(self, x):
   x, _status = self.lstm(x)
   x = self.fc(x[:, -1])
   return x

In [408]:
#모델 생성하는 데 사용하는 device 에 따라서 생성
LSTM = LSTM(data_dim, hidden_dim, seq_length, output_dim,1).to(device)

- 파이썬 클래스로 LSTM model 을 설계, pytorch nn 의 모델을 상속함
- forward 함수와 init이 핵심이고, reset_hidden_state 는 은닉층을 초기화하기 위해서 추가한 함수

### 4. Training/Test

In [409]:
from math import e
#train code
def train_model(model, train_df, epochs=None, Ir=None, verbose=10, patience=10):

  #손실함수
  criterion = nn.MSELoss().to(device)

  #최적화함수
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  train_hist = np.zeros(epochs)
  for epoch in range(epochs):
    avg_cost = 0
    total_batch = len(train_df)

    for batch_idx, samples in enumerate(train_df):
      x_train, y_train = samples
      #seq 별 hidden state reset
      model.reset_hidden_state()

      #H(x) 계산
      outputs = model(x_train)
      #cost 계산
      loss = criterion(outputs, y_train)
      #cost로 H(x) 개선
      '''
      파이토치에서는 gradients 를 항상 0으로 만들어주어야 하고
      그렇지 않으면 gradient 가 의도한 방향과 다른 방향을 가리켜 학습이 잘 이루어지지 않는다.
      '''
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      avg_cost += loss / total_batch
    train_hist[epoch] = avg_cost

    if epoch % verbose == 0:
      print('Epoch:', '%04d' % (epoch + 1), 'train_less : ', '{:.4f}'.format(avg_cost))

    if(epoch%patience==0) & (epoch!=0):
      if train_hist[epoch-patience] < train_hist[epoch]:
        print('\n Early Stopping')
        break
  return model.eval(), train_hist

model, train_hist = train_model(LSTM, dataloader, epochs=epochs, Ir=learning_rate, verbose=20, patience=10)

Epoch: 0001 train_less :  0.0000
Epoch: 0021 train_less :  0.0000
Epoch: 0041 train_less :  0.0000
Epoch: 0061 train_less :  0.0000
Epoch: 0081 train_less :  0.0000
Epoch: 0101 train_less :  0.0000
Epoch: 0121 train_less :  0.0000
Epoch: 0141 train_less :  0.0000
Epoch: 0161 train_less :  0.0000
Epoch: 0181 train_less :  0.0000
Epoch: 0201 train_less :  0.0000
Epoch: 0221 train_less :  0.0000
Epoch: 0241 train_less :  0.0000
Epoch: 0261 train_less :  0.0000
Epoch: 0281 train_less :  0.0000
Epoch: 0301 train_less :  0.0000
Epoch: 0321 train_less :  0.0000
Epoch: 0341 train_less :  0.0000
Epoch: 0361 train_less :  0.0000
Epoch: 0381 train_less :  0.0000
Epoch: 0401 train_less :  0.0000
Epoch: 0421 train_less :  0.0000
Epoch: 0441 train_less :  0.0000
Epoch: 0461 train_less :  0.0000
Epoch: 0481 train_less :  0.0000


In [ ]:
#evaluation
with torch.no_grad():
  pred = []
  for pr in range(len(testX_tensor)):
    model.reset_hidden_state()

    predicted = model(torch.unsqueeze(testX_tensor[pr], 0))
    predicted = torch.flatten(predicted)
    pred.append(predicted.item())

In [ ]:
scaler

In [344]:
testY_tensor

tensor([[0.0000],
        [0.0000],
        [0.0000],
        ...,
        [0.1825],
        [0.1588],
        [0.1406]], device='cuda:0')

In [345]:
#inverse : 스캐일링했던 값들을 원래대로 돌려놓는 과정
#pred_inverse = scaler.inverse_transform(np.array(pred).reshape(-1, 2))
#testY_inverse = scaler.inverse_transform(testY_tensor.cpu().numpy().reshape(-1, 2))

In [346]:
def MAE(true, pred):
  return np.mean(np.abs(true-pred))
print(f'MAE : {MAE(pred, testY)}')


def RMSE(true, pred):
  return np.sqrt(np.mean((true-pred)**2))
print(f'RMSE : {RMSE(pred, testY)}')

MAE : 0.08016225919601812
RMSE : 0.15855614889152322


In [347]:
#target test
